# Spacy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
# !pip install -U spacy[cuda118]

In [ ]:
import spacy
gpu = spacy.prefer_gpu()
print('GPU:', gpu)

GPU: True


In [ ]:
src = '/content/drive/Othercomputers/My Computer/Masters/Rice/Spring23/COMP 642/COMP 642 Project/Code/Models/NER/Spacy'

import sys
sys.path.append(src)

In [ ]:
%cd '/content/drive/Othercomputers/My Computer/Masters/Rice/Spring23/COMP 642/COMP 642 Project/Code/Models/NER/Spacy'

/content/drive/Othercomputers/My Computer/Masters/Rice/Spring23/COMP 642/COMP 642 Project/Code/Models/NER/Spacy


In [ ]:
import json
import traceback
def get_entites_spacy(file_name):
    """Extract the entities for NER traineing from Label Studio json data format for scipy fine tuning.

    Args:
        file_name (str): JSON File location.

    Returns:
        list: Extracted entites in the format [(text, {"entities":[(start, end, label)]})]
    """
    # labels = ['Organization', 'Asset', 'Person', 'Investor']
    esg_map = {'Soc': 'Social', 'Env': 'Environmental', 'Gov': 'Governance'}
    documents = []
    # Open the JSONL file for reading
    with open(file_name, 'r', encoding='utf-8') as jsonl_file:
        # Iterate over the lines in the file
        data = json.load(jsonl_file)
        for item in data:
            text = item['data']['text']

            entities = []
            # Parse each line as a JSON object
            for entry in item['annotations']:
                for res in entry['result']:
                    try:
                        boundaries = []
                    
                        if res['type'] == 'labels':
                            value = res['value']
                            from_name = res['from_name']
                            ner = from_name.split('-')[1]
                            
                            if ner == 'General':
                                continue
                                if value['labels'][0] in labels:
                                    ner = value['labels'][0]
                            else:
                                ner = esg_map[ner]
                            boundaries.append(value['start'])
                            boundaries.append(value['end'])
                            entities.append((boundaries[0], boundaries[1], ner))
                    except Exception as e:
                        print(res)
                        print(traceback.print_exc())
                        break
            
            documents.append({'text': text, 'entities': entities})
    return documents

In [ ]:
# import extract_data as ex

documents = []
documents += get_entites_spacy('chev_data.json')
documents += get_entites_spacy('chev_data2.json')

In [ ]:
from google.colab import files
files.upload()

In [ ]:
import json
import re

with open('./ner_data_spacy.txt', 'r') as f:
    data = json.load(f)


def entity_boundaries(data):
    #df_data = pd.DataFrame(columns=['data'])
    lst_data = []
    for idx in range(len(data)):
        text = data[idx]['text']
        entities = data[idx]['entities']
        ent_w_loc = []

        for entity in entities:
            if entity['entity'] not in ['Environmental', 'Social', 'Governance']:
              continue
            pattern = re.compile(entity['value'])
            m_val = pattern.search(text)
            if m_val is not None:
                ent_w_loc.append((m_val.start(), m_val.end(), entity['entity']))
                
                
        if len(ent_w_loc)!=0:
          #df_data = pd.concat([df_data, pd.DataFrame(
          #    {'data':[(text, {'entities':ent_w_loc})]})], ignore_index=True)
            lst_data.append(({'text': text, 'entities':ent_w_loc}))

    return lst_data

lst_data=entity_boundaries(data)
len(lst_data)

180

In [ ]:
lst_data[100]

{'text': 'The energy company Enbridge faced legal action and public criticism for their role in the Line 5 pipeline controversy and lack of transparency.',
 'entities': [(122, 142, 'Governance')]}

In [ ]:
documents += lst_data

In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
doc_bin = DocBin()

In [ ]:
len(documents)

186

In [ ]:
from spacy.util import filter_spans
training_data = documents
for training_example  in tqdm(training_data[:150]): 
    # print(training_example['text'])
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

doc_bin.to_disk("train.spacy")

100%|██████████| 150/150 [00:00<00:00, 1093.03it/s]


In [ ]:
from spacy.util import filter_spans
training_data = documents
for training_example  in tqdm(training_data[150:]): 
    # print(training_example['text'])
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

doc_bin.to_disk("val.spacy")

100%|██████████| 36/36 [00:00<00:00, 3208.29it/s]


In [ ]:
import locale
print(locale.getpreferredencoding())

ANSI_X3.4-1968


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install spacy-transformers -qq

In [ ]:
!python -m spacy init fill-config ./base_config.cfg config.cfg 

2023-05-03 04:27:25.904004: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
%ls

 base_config.cfg  'Copy of NER.ipynb'   models_05_02/             output/
 chev_data2.json   extract_data.py     'ner_data_spacy (1).txt'   __pycache__/
 chev_data.json    model-best/          ner_data_spacy.txt        train.spacy
 config.cfg        model-last/          NER.ipynb                 val.spacy


In [ ]:
!python3 -m spacy download en_core_web_lg

2023-05-03 04:28:59.961283: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 3.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
!python -m spacy train config.cfg --gpu-id 0 --output ./models_05_02 --paths.train ./train.spacy --paths.dev ./val.spacy 

2023-05-03 04:29:55.790792: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: models_05_02
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-05-03 04:30:00,087] [INFO] Set up nlp object from config
[2023-05-03 04:30:00,103] [INFO] Pipeline: ['transformer', 'ner']
[2023-05-03 04:30:00,106] [INFO] Created vocabulary
[2023-05-03 04:30:00,108] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT exp

In [ ]:
import spacy
nlp_ner = spacy.load("./models_05_02/model-last")

In [ ]:
doc = nlp_ner("Some companies are investing in green technologies and reducing their environmental impact.")

colors = {"Social": "#F67DE3", "Governance": "#7DF6D9", "Environmental":"#a6e22d"}
options = {"colors": colors} 

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

In [ ]:
!python -m spacy package "./models_05_02/model-last" "./models_05_02/output_last" --build wheel

ℹ Building package artifacts: wheel

error in en_ner-esg setup command: 'NoneType' object has no attribute 'group'



✔ Including 1 package requirement(s) from meta and config
spacy-transformers>=1.2.3,<1.3.0
✔ Loaded meta.json from file
models_05_02\model-last\meta.json
✔ Generated README.md from meta.json
✔ Successfully created package directory 'en_ner-esg-0.0.1'
models_05_02\output_last\en_ner-esg-0.0.1


In [ ]:
# import shutil
# shutil.rmtree('./models_05_02/output_last/')

In [ ]:
!huggingface-cli login

^C


In [1]:
!pip install https://huggingface.co/TrajanovRisto/en_ner_esg/resolve/main/en_ner_esg-any-py3-none-any.whl

# Using spacy.load().
import spacy
nlp = spacy.load("en_ner_esg")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.8/426.8 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.7/190.7 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 68.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.3 MB/s eta 0:00:00


In [10]:
doc = nlp("These concerns reflect a converging consensus by policymakers, investors, and companies on growing climate risk, the need to limit global temperature increase to 1.5° C (net zero global greenhouse gas (GHG) emissions by 2050),5 and the impact of such actions to companies.")

colors = {"Social": "#F67DE3", "Governance": "#7DF6D9", "Environmental":"#a6e22d"}
options = {"colors": colors} 

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)